In [1]:
from py2neo import Graph, Node, Relationship

In [4]:
g=Graph("bolt://localhost:7687",password="1234")

In [5]:
g.run("MATCH (n) DETACH DELETE n")
#g.run("MATCH(n) WITH n LIMIT 500000 DETACH DELETE n RETURN count(*);")
#g.run("CALL apoc.schema.assert({}, {})")
#g.run("MATCH ()-[r:]->() DELETE r")

# CREATING PLAYER NODES

In [6]:
query1= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (player:Player{name:row.Name})
ON CREATE SET player.position = row.Position, player.Image = row.Player_image
'''

In [7]:
g.run(query1)

# CREATING COUNTRY NODES

In [8]:
query2= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (country:Country{name:row.Player_nationality})
'''

In [9]:
g.run(query2)

# CREATING CLUB NODES

In [10]:
query3= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (club:Club{name:row.Team_from})
'''

In [11]:
g.run(query3)

In [12]:
query4= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (club:Club{name:row.Team_to})
'''

In [13]:
g.run(query4)

# CREATING LEAGUES NODES

In [14]:
query5= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (league:League{name:row.League_from})
'''

In [15]:
g.run(query5)

In [16]:
query6= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (league:League{name:row.League_to})
'''

In [17]:
g.run(query6)

# CREATING TRANSFER NODES

In [20]:
query= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (transfer:Transfer{
    id: row.ID,
    fee: row.Transfer_fee
    })
'''

In [21]:
g.run(query)

# CREATING SEASON NODES

In [22]:
queryS= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (season:Season{year: row.Season   })
'''

In [23]:
g.run(queryS)

# CREATING ROLE NODES

In [24]:
queryR= '''
USING PERIODIC COMMIT
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS row
MERGE (role:Role{position: row.Position})
'''

In [25]:
g.run(queryR)

# CREATING NATIONALITY EDGES

In [26]:
query7 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (pl:Player), (co:Country)
WHERE pl.name = csvLine.Name AND co.name = csvLine.Player_nationality
MERGE (pl) -[r:nationality]-> (co) 
'''

In [27]:
g.run(query7)

# CREATING PLAYS_IN EDGES

In [28]:
query8 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (cl:Club), (le:League)
WHERE cl.name = csvLine.Team_from AND le.name = csvLine.League_from
MERGE (cl) -[r:plays_in]-> (le) 
'''

In [31]:
g.run(query8)

In [32]:
query9 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (cl:Club), (le:League)
WHERE cl.name = csvLine.Team_to AND le.name = csvLine.League_to
MERGE (cl) -[r:plays_in]-> (le) 
'''

In [33]:
g.run(query9)

# CREATING TRANSFER -> TEAMFROM/TO EDGES

In [34]:
query10 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (tr:Transfer), (cl:Club)
WHERE tr.id = csvLine.ID AND cl.name = csvLine.Team_from
MERGE (tr) -[r:teamFrom]-> (cl) 
'''

In [35]:
g.run(query10)

In [36]:
query11 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (tr:Transfer), (cl:Club)
WHERE tr.id = csvLine.ID AND cl.name = csvLine.Team_to
MERGE (tr) -[r:teamTo]-> (cl) 
'''

In [37]:
g.run(query11)

# CREATING TRANSFER -> PLAYER_INVOLVED EDGES

In [38]:
query12 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (tr:Transfer), (pl:Player)
WHERE tr.id = csvLine.ID AND pl.name = csvLine.Name
MERGE (tr) -[r:playerInvolved]-> (pl) 
SET r.Age=csvLine.Age, r.MarketValue = csvLine.Market_value
'''

In [39]:
g.run(query12)

# CREATING PLAYER -> PLAYSAS EDGES

In [48]:
query13 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (pl:Player), (ro:Role)
WHERE pl.name = csvLine.Name AND ro.position = csvLine.Position
MERGE (pl) -[r:playsAs]-> (ro) 
'''

In [49]:
g.run(query13)

# CREATING TRANSFER -> TRANSFERSEASON EDGES

In [50]:
query14 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (tr:Transfer), (se:Season)
WHERE tr.id = csvLine.ID AND se.year = csvLine.Season
MERGE (tr) -[r:transferSeason]-> (se)
'''

In [51]:
g.run(query14)

# ------- transfer as Edge ------- #

In [46]:
query15 = '''
LOAD CSV WITH HEADERS FROM "file:///transfers.csv" AS csvLine
MATCH (cl1:Club), (cl2:Club)
WHERE cl1.name = csvLine.Team_from AND cl2.name = csvLine.Team_to
MERGE (cl1) -[r:transfer]-> (cl2) 
SET r.fee=csvLine.Transfer_fee, r.type= csvLine.type, r.season= csvLine.Season, r.player = csvLine.Name, r.Market_value = csvLine.Market_value
'''

In [47]:
g.run(query15)

In [ ]:
MATCH (c2:Club)-[tr:transfer]->(c1:Club)<-[tto:teamTo]-(t:Transfer)-[ts:transferSeason]->(s:Season)

RETURN s.year, c1.name, COUNT(ts) AS numberOfBuys
ORDER BY numberOfBuys DESC